## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Puerto Ayora,EC,2021-12-18 22:02:48,-0.7393,-90.3518,72.55,89,84,4.00
1,1,Souillac,MU,2021-12-18 22:02:49,-20.5167,57.5167,74.35,88,20,3.44
2,2,Rikitea,PF,2021-12-18 22:02:49,-23.1203,-134.9692,77.54,76,85,7.38
3,3,Albany,US,2021-12-18 22:00:09,42.6001,-73.9662,36.81,91,100,1.01
4,4,Hilo,US,2021-12-18 22:02:50,19.7297,-155.0900,82.27,15,13,7.85


In [17]:
city_data_df.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
count,698.00000,698.000000,698.000000,698.000000,698.000000,698.000000,698.000000
mean,348.50000,19.482985,18.630738,51.385415,74.770774,61.681948,7.849026
std,201.63953,32.894150,86.638237,30.225823,20.229647,39.453719,6.062235
min,0.00000,-54.800000,-175.200000,-50.080000,12.000000,0.000000,0.000000
25%,174.25000,-8.079350,-56.400150,35.130000,67.000000,20.000000,3.205000
50%,348.50000,24.621700,22.407300,61.735000,78.000000,79.000000,6.420000
75%,522.75000,46.472525,92.490375,75.172500,90.000000,99.000000,10.360000
max,697.00000,78.218600,178.416700,98.490000,100.000000,100.000000,32.410000


In [18]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     698 non-null    int64  
 1   City        698 non-null    object 
 2   Country     691 non-null    object 
 3   Date        698 non-null    object 
 4   Lat         698 non-null    float64
 5   Lng         698 non-null    float64
 6   Max Temp    698 non-null    float64
 7   Humidity    698 non-null    int64  
 8   Cloudiness  698 non-null    int64  
 9   Wind Speed  698 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 54.7+ KB


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
lowest_temp = float(input("What is lowest temperature you would accept for your vacation? "))
highest_temp = float(input("What is lowest temperature you would accept for your vacation? "))


What is lowest temperature you would accept for your vacation? 75
What is lowest temperature you would accept for your vacation? 85


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
matching_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= highest_temp) &
                                       (city_data_df["Max Temp"] >= lowest_temp)]
matching_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Rikitea,PF,2021-12-18 22:02:49,-23.1203,-134.9692,77.54,76,85,7.38
4,4,Hilo,US,2021-12-18 22:02:50,19.7297,-155.0900,82.27,15,13,7.85
7,7,Avarua,CK,2021-12-18 22:02:51,-21.2078,-159.7750,82.45,69,75,13.80
8,8,Sambava,MG,2021-12-18 22:02:52,-14.2667,50.1667,76.10,90,16,4.83
13,13,Axim,GH,2021-12-18 22:02:54,4.8699,-2.2405,80.42,84,62,8.48


In [10]:
# 4a. Determine if there are any empty rows.
matching_cities_df.isnull().sum()

City_ID       0
City          0
Country       1
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
matching_cities_df = matching_cities_df.dropna()
matching_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Rikitea,PF,2021-12-18 22:02:49,-23.1203,-134.9692,77.54,76,85,7.38
4,4,Hilo,US,2021-12-18 22:02:50,19.7297,-155.0900,82.27,15,13,7.85
7,7,Avarua,CK,2021-12-18 22:02:51,-21.2078,-159.7750,82.45,69,75,13.80
8,8,Sambava,MG,2021-12-18 22:02:52,-14.2667,50.1667,76.10,90,16,4.83
13,13,Axim,GH,2021-12-18 22:02:54,4.8699,-2.2405,80.42,84,62,8.48


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# hotel_df = matching_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# , "Current Description" was removed because it in not in the data
hotel_df = matching_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Rikitea,PF,77.54,-23.1203,-134.9692,
4,Hilo,US,82.27,19.7297,-155.0900,
7,Avarua,CK,82.45,-21.2078,-159.7750,
8,Sambava,MG,76.10,-14.2667,50.1667,
13,Axim,GH,80.42,4.8699,-2.2405,
14,Simoes,BR,83.43,-7.5989,-40.8178,
15,Buala,SB,81.72,-8.1450,159.5921,
19,Kapaa,US,78.78,22.0752,-159.3190,
23,Namatanai,PG,79.07,-3.6667,152.4333,
27,Makakilo City,US,80.92,21.3469,-158.0858,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.

    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
